# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to nie209@uregina.ca and will expire on December 26, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1536371094.log


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [4]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [5]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [37]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 0.740213     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 0.792891     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [55]:
x = model_all['coefficients']['name','value']
x = x[x['value'] >0]
print x

+------------------+---------------+
|       name       |     value     |
+------------------+---------------+
|   (intercept)    |  274873.05595 |
|    bathrooms     | 8468.53108691 |
|   sqft_living    | 24.4207209824 |
| sqft_living_sqrt | 350.060553386 |
|      grade       | 842.068034898 |
|    sqft_above    | 20.0247224171 |
+------------------+---------------+
[? rows x 2 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use sf.materialize() to force materialization.


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [80]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [81]:
import numpy as np
output_dict ={}
l1_penalty_list = np.logspace(1,7, num=13)
for l1 in l1_penalty_list:
    model = graphlab.linear_regression.create(training, target='price', l2_penalty=0, l1_penalty=l1,
                                              verbose=False, validation_set=None, features=all_features)
    predictions = model.predict(validation)
    residual = predictions - validation['price']
    Rss = sum(residual*residual)
    output_dict[l1] = Rss
print output_dict

{100000.0: 625767302791633.4, 316227.76601683791: 625769507643885.1, 100.0: 625766286057887.0, 3162.2776601683795: 625766317206077.8, 31.622776601683793: 625766285362395.2, 1000.0: 625766295212186.0, 10.0: 625766285142461.2, 31622.776601683792: 625766606749281.4, 10000.0: 625766386760661.5, 3162277.6601683795: 625799062845466.9, 316.22776601683796: 625766288257224.9, 1000000.0: 625776517727025.8, 10000000.0: 625883719085424.5}


In [82]:
for key, values in output_dict.items():
    if values == min(output_dict.values()):
        print key, values

10.0 6.25766285142e+14


*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [83]:
model = graphlab.linear_regression.create(training, verbose=False, validation_set=None,
                                          target="price", features= all_features,
                                          l1_penalty=10.0, l2_penalty=0.)
predictions = model.predict(testing)
residual = testing['price'] - predictions
rss = sum(residual* residual)
print rss

1.56983602382e+14


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [86]:
x = model['coefficients']['name','value']
x = x[x['value'] >0]
print x.print_rows(20)

+------------------+------------------+
|       name       |      value       |
+------------------+------------------+
|   (intercept)    |  18993.4272128   |
|     bedrooms     |  7936.96767903   |
| bedrooms_square  |  936.993368193   |
|    bathrooms     |  25409.5889341   |
|   sqft_living    |  39.1151363797   |
| sqft_living_sqrt |  1124.65021281   |
|     sqft_lot     | 0.00348361822299 |
|  sqft_lot_sqrt   |  148.258391011   |
|      floors      |   21204.335467   |
|  floors_square   |  12915.5243361   |
|    waterfront    |  601905.594545   |
|       view       |  93312.8573119   |
|    condition     |  6609.03571245   |
|      grade       |  6206.93999188   |
|    sqft_above    |  43.2870534193   |
|  sqft_basement   |  122.367827534   |
|     yr_built     |  9.43363539372   |
|   yr_renovated   |  56.0720034488   |
+------------------+------------------+
[18 rows x 2 columns]

None


# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [87]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [93]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [98]:
weight_list = []
nonzero_dict = {}
for l1 in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                             l1_penalty=l1, l2_penalty=0,
                                             verbose=None, validation_set=None)
    weight_list.append(model['coefficients']['value'])
    nonzero_dict[l1] = model['coefficients']['value'].nnz()
        

In [119]:
import pprint

pprint.pprint(nonzero_dict)

{100000000.0: 18,
 127427498.57031322: 18,
 162377673.91887242: 18,
 206913808.11147901: 18,
 263665089.87303555: 17,
 335981828.62837881: 17,
 428133239.8719396: 17,
 545559478.11685145: 17,
 695192796.17755914: 17,
 885866790.41008317: 16,
 1128837891.6846883: 15,
 1438449888.2876658: 15,
 1832980710.8324375: 13,
 2335721469.0901213: 12,
 2976351441.6313133: 10,
 3792690190.7322536: 6,
 4832930238.5717525: 5,
 6158482110.6602545: 3,
 7847599703.5146227: 1,
 10000000000.0: 1}


['__class__',
 '__cmp__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'copy',
 'fromkeys',
 'get',
 'has_key',
 'items',
 'iteritems',
 'iterkeys',
 'itervalues',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values',
 'viewitems',
 'viewkeys',
 'viewvalues']

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [124]:
l1_penalty_min = nonzero_dict.keys()[0]
l1_penalty_max = l1_penalty_min
current_value = max_nonzeros
for key, value in nonzero_dict.items():
    
    if value > max_nonzeros:
        if l1_penalty_min < key:
            l1_penalty_min = key
            
            
## dont know figure out the logic for finding the l1_penalty_max
print l1_penalty_min
l1_penalty_max = 3792690190.7322536
print l1_penalty_max

2976351441.63
3792690190.73


***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [135]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)
print l1_penalty_values

[  2.97635144e+09   3.01931664e+09   3.06228184e+09   3.10524703e+09
   3.14821223e+09   3.19117743e+09   3.23414263e+09   3.27710782e+09
   3.32007302e+09   3.36303822e+09   3.40600341e+09   3.44896861e+09
   3.49193381e+09   3.53489901e+09   3.57786420e+09   3.62082940e+09
   3.66379460e+09   3.70675980e+09   3.74972499e+09   3.79269019e+09]


* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [137]:
output ={}
for l1 in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              l1_penalty=l1, l2_penalty=0, 
                                              validation_set=None, verbose=None)
    prediction = model.predict(validation)
    residual = prediction - validation['price']
    rss = sum(residual* residual)
    output[l1] = rss, model['coefficients']['value'].nnz()
    

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [145]:
for key, value in output.items():
    if value[1] == max_nonzeros:
        print key, value

3577864204.13 (1060799531763290.2, 7)
3448968612.16 (1046937488751713.6, 7)
3491933809.48 (1051147625612863.0, 7)
3534899006.81 (1055992735342999.2, 7)


In [147]:
model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                          l1_penalty=3448968612.16, l2_penalty=0, 
                                          validation_set=None, verbose=None)


In [149]:
x  = model['coefficients']['name', 'value']
x = x[x['value']>0]
print x.print_rows(20)

+------------------+---------------+
|       name       |     value     |
+------------------+---------------+
|   (intercept)    | 222253.192544 |
|     bedrooms     | 661.722717782 |
|    bathrooms     | 15873.9572593 |
|   sqft_living    | 32.4102214513 |
| sqft_living_sqrt | 690.114773313 |
|      grade       | 2899.42026975 |
|    sqft_above    | 30.0115753022 |
+------------------+---------------+
[7 rows x 2 columns]

None
